The deep learning based model

In [ ]:
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install keras

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_csv('WELFake_Dataset.csv', engine='python', on_bad_lines='skip')
title = df[df.columns[0]].values
text = df[df.columns[1]].values
label_col = df[df.columns[-1]]

valid_labels_mask = label_col.apply(lambda x: isinstance(x, str) and x.isdigit() and (x == '0' or x == '1'))
df_filtered = df[valid_labels_mask].copy()
title = df_filtered[df_filtered.columns[0]].values
text = df_filtered[df_filtered.columns[1]].values
label = df_filtered[df_filtered.columns[-1]].values.astype(int) # Convert label to integer
title = [str(item) if pd.notna(item) else '' for item in title]
text = [str(item) if pd.notna(item) else '' for item in text]

title_train, title_test, text_train, text_test, label_train, label_test = train_test_split(title, text, label, test_size=0.3, random_state=84)

tokenizer1 = Tokenizer(num_words=100)
tokenizer1.fit_on_texts(title_train)
title_train = tokenizer1.texts_to_sequences(title_train)
title_test = tokenizer1.texts_to_sequences(title_test)
tokenizer2 = Tokenizer(num_words=30000)
tokenizer2.fit_on_texts(text_train)
text_train = tokenizer2.texts_to_sequences(text_train)
text_test = tokenizer2.texts_to_sequences(text_test)
title_train_pad = pad_sequences(title_train, maxlen=100, padding="post")
title_test_pad = pad_sequences(title_test, maxlen=100, padding="post")
text_train_pad = pad_sequences(text_train, maxlen=3000, padding="post")
text_test_pad = pad_sequences(text_test, maxlen=3000, padding="post")

title_input = Input(shape=(100,), name='title_input')
text_input = Input(shape=(3000,), name='text_input')
title_embedding = Embedding(input_dim=10000, output_dim=128, input_length=100)(title_input)
title_lstm = LSTM(64)(title_embedding)
text_embedding = Embedding(input_dim=30000, output_dim=128, input_length=3000)(text_input)
text_lstm = LSTM(64)(text_embedding)
merged = Concatenate()([title_lstm, text_lstm])
dense = Dense(64, activation='relu')(merged)
output = Dense(1, activation='sigmoid')(dense)
model = tf.keras.Model(inputs=[title_input, text_input], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit([title_train_pad, text_train_pad], label_train, epochs=30, batch_size=32, validation_data=([title_test_pad, text_test_pad], label_test))

test_loss, test_acc = model.evaluate([title_test_pad, text_test_pad], label_test)
print('Test accuracy:', test_acc)

The LLM based model (Mistral 7B)

In [ ]:
!pip install bitsandbytes-cuda110 bitsandbytes
!pip install --upgrade transformers
!pip install evaluate
from huggingface_hub import login
login(new_session=False)

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from datasets import Dataset
import evaluate
import pandas as pd
import torch

model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True,
    num_labels=2,
    torch_dtype=torch.float16
)
model.config.pad_token_id = tokenizer.pad_token_id
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
df = pd.read_csv("WELFake_Dataset.csv")
dataset = Dataset.from_pandas(df)
def preprocess(example):
    text = f"Title: {example['title']}\nText: {example['text']}"
    encoding = tokenizer(text, truncation=True, max_length=512)
    encoding["label"] = int(example["label"])
    return encoding

tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

data_collator = DataCollatorWithPadding(tokenizer)
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return accuracy.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./mistral-fake-news-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    save_total_limit=2,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    gradient_checkpointing=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

trainer.train()
